## Load/import packages

In [1]:
import json
import time
import scipy
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import tensorflow.keras.backend as K
import kerastuner as kt

from tensorflow.keras import Sequential, layers
from tensorflow.keras.layers import Layer
from tensorflow.keras.losses import CategoricalCrossentropy
from sklearn.utils import class_weight

from os import listdir
from os.path import join, splitext, normpath

# Import modules to run custom FW-RNN cell
from tensorflow.python.keras.layers.recurrent import (
    _generate_zero_filled_state_for_cell,
    _generate_zero_filled_state,
    ops,
    tensor_shape,
    activations,
    initializers,
    regularizers,
    nest,
    array_ops,
)

# Import variables and functions from my own scripts
from functions import plot_history, arr_replacevalue
from load_features import (
    train_features_AW2,
    val_features_AW2,
    train_labels_AW2,
    val_labels_AW2,
    labels_reshaper,
    features_reshaper,
)

%matplotlib inline

# Limit GPU memory usage
for device in tf.config.experimental.list_physical_devices("GPU"):
    tf.config.experimental.set_memory_growth(device, True)

# Prepare data

In [2]:
# Reshape data to specified sequence length
length = 60
seq_train_features = features_reshaper(train_features_AW2, length) # divisible 13, 39, 197
seq_val_features = features_reshaper(val_features_AW2, length)

seq_train_labels = labels_reshaper(train_labels_AW2, length)
seq_val_labels = labels_reshaper(val_labels_AW2, length)

In [3]:
def comp_sampleweights(labels):
    # Convert one-hot encoded labels back to label integers
    train_label_ints = np.argmax(labels, axis=2)

    # Compute class weights with sklearn
    class_weights = class_weight.compute_class_weight(
        "balanced", np.unique(train_label_ints), train_label_ints.flatten()
    )
    d_class_weights = dict(enumerate(class_weights))

    # Pass a 2D array with shape (samples, sequence_length), to apply a different weight to every timestep of every sample
    return arr_replacevalue(train_label_ints, d_class_weights)
    
train_samples_weights = comp_sampleweights(seq_train_labels) 

C:\Users\ion\anaconda3\envs\tf-gpu\lib\site-packages\sklearn\utils\validation.py:70: FutureWarning: Pass classes=[0 1 2 3 4 5 6], y=[0 0 0 ... 6 6 6] as keyword args. From version 0.25 passing these as positional arguments will result in an error
  FutureWarning)


# Create FW-RNN Cell
-  Build custom FW_RNN cell and wrap it in RNN layer (https://www.tensorflow.org/api_docs/python/tf/keras/layers/RNN), like this: RNN(FW_RNN)
    -  "The cell abstraction, together with the generic keras.layers.RNN class, make it very easy to implement custom RNN architectures for your research."

Created by using this guide: https://www.tensorflow.org/guide/keras/custom_layers_and_models

In [11]:
class FW_RNNCell(layers.Layer):
    def __init__(
        self,
        units,
        use_bias,
        batch_size,
        decay_rate,
        learning_rate,
        activation,
        step,
        LN = layers.LayerNormalization(),
        **kwargs
        
    ):
        super(FW_RNNCell, self).__init__(**kwargs)
        self.units = units
        self.step = step
        self.use_bias = use_bias
        self.activation = activations.get(activation)
        self.l = decay_rate
        self.e = learning_rate
        self.LN = LN
        

        self.batch = batch_size
        self.state_size = self.units

        # Initializer & regularizer for the slow input-to-hidden weights matrix
        self.C_initializer = initializers.get("glorot_uniform")

        # Initializer & regularizer for the slow hidden weights matrix
        self.W_h_initializer = initializers.get("identity")

        # Initializer & regularizer for the fast weights matrix
        self.A_initializer = initializers.get("zeros")

        # Initializer for the bias vector.
        self.b_x_initializer = initializers.get("zeros")
        
        
    def build(self, input_shape):
        # Build is only called at the start, to initialize all the weights and biases

        # C = Slow input-to-hidden weights [shape (4608, 64)]
        self.C = self.add_weight(
            shape=(input_shape[-1], self.units),
            name="inputweights",
            initializer=self.C_initializer,
        )
        
        # W_h The previous hidden state via the slow transition weights [shape (units, units)]
        # they suggest to multiply it with 0.05, so gain = 0.05
        self.W_h = self.add_weight(
            shape=(self.units, self.units),
            name="hiddenweights",
            initializer=self.W_h_initializer, 
        )
        self.W_h = tf.scalar_mul(0.05, self.W_h)
        
        # A (fast weights) [shape (batch_size, units, units)]
        self.A = self.add_weight(
            shape=(self.batch, self.units, self.units),
            name="fastweights",
            initializer=self.A_initializer,
        )
        
        if self.use_bias:
            self.bias = self.add_weight(
                shape=(self.units,), name="bias", initializer=self.b_x_initializer,
            )
        else:
            self.bias = None
        self.built = True
        
    def call(self, inputs, states, training=None):
        prev_output = states[0] if nest.is_sequence(states) else states
        
        # Next hidden state h(t+1) is computed in two steps:
        # Step 1 calculate preliminary vector: h_0(t+1) = f(W_h ⋅ h(t) + C ⋅ x(t))
        h = K.dot(prev_output, self.W_h) + K.dot(inputs, self.C)
        if self.bias is not None:
            h = h + self.bias
        if self.activation is not None:
            h = self.activation(h)
        
        # Reshape h to use with a
        h_s = tf.reshape(h, [self.batch, 1, self.units])
        
        # Define preliminary vector in variable
        prelim = tf.reshape(K.dot(prev_output, self.W_h), (h_s.shape)) + tf.reshape(K.dot(inputs, self.C), (h_s.shape))
        
        # Fast weights update rule: A(t) = λ*A(t-1) + η*h(t) ⋅ h(t)^T
        self.A.assign(
            tf.math.add(
                tf.scalar_mul(self.l, self.A),
                tf.scalar_mul(
                    self.e, tf.linalg.matmul(tf.transpose(h_s, [0, 2, 1]), h_s)
                ),
            )
        )
        
        # Step 2: Initiate inner loop with preliminary vector, which runs for S steps
        # to progressively change the hidden state into h(t+1) = h_s(t+1)
        # h_s+1(t+1) f([W_h ⋅ h(t) + C ⋅ x(t)]) + A(t)h_s(t+1)
        for _ in range(self.step):
            h_s = tf.math.add(prelim, tf.linalg.matmul(h_s, self.A)) 
            if self.bias is not None:
                h_s = h_s + self.bias
            if self.activation is not None:
                h_s = self.activation(h_s)
            
            # Apply layer normalization on hidden state
            h_s = self.LN(h_s)

        h = tf.reshape(h_s, [self.batch, self.units])

        output = h
        new_state = [output] if nest.is_sequence(states) else output
        return output, new_state

    def get_initial_state(self, inputs=None, batch_size=None, dtype=None):
        return _generate_zero_filled_state_for_cell(self, inputs, batch_size, dtype)

# Build model

In [15]:
# Build model with sequential api
def build_model(batch, activation_function):
    model = Sequential()
    model.add(tf.keras.Input(shape=(seq_train_features.shape[1], seq_train_features.shape[2])))
    model.add(
        layers.RNN(
            FW_RNNCell(units=128, 
                       use_bias=True, 
                       activation=activation_function, 
                       step=1,
                       decay_rate = 0.95,
                       learning_rate = 0.5,
                       batch_size=batch,
                   ),
            return_sequences=True,
            name = 'FW-RNN'
        )
    )
    model.add(layers.Dense(7, activation="softmax", name="Dense_Output"))
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
        loss=CategoricalCrossentropy(label_smoothing=0.1),
        metrics=["accuracy"],
        run_eagerly=False,
    )
    return model


# Train + Evaluate model

In [16]:
# from sklearn.model_selection import KFold
# for train, test in kfold.split()

In [ ]:
batchsize = 32
a = (seq_train_features.shape[0] // batchsize) * batchsize
b = (seq_val_features.shape[0] // batchsize) * batchsize


for act in ['relu', 'tanh']:
    fw_rnn = build_model(batchsize, act)
    fw_rnn.summary()

    # Access tensorboard in cmd of the main repo folder with following code:
    # tensorboard --logdir='logs/'
    name = 'FW-RNN_1layer64_rmsprop_identitymatrix0.05_{}_{}'.format(act, int(time.time()))
    tb_callback = tf.keras.callbacks.TensorBoard(log_dir='logs/{}'.format(name))

    # Set callbacks for model training
#     csvlog = tf.keras.callbacks.CSVLogger(
#         "data/models/FW-RNN_performance.csv", separator=",", append=False
#     )

    history_best = fw_rnn.fit(
        seq_train_features[:a],
        seq_train_labels[:a],
        batch_size=batchsize,
        sample_weight=train_samples_weights[:a],
        validation_data=(
            seq_val_features[:b],
            seq_val_labels[:b],
        ),
        callbacks=[
#                 csvlog, 
#                 tb_callback
        ],
        epochs=150,
        verbose=2,
        shuffle=True,
    )

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
FW-RNN (RNN)                 (32, 60, 128)             1114496   
_________________________________________________________________
Dense_Output (Dense)         (32, 60, 7)               903       
Total params: 1,115,399
Trainable params: 1,115,399
Non-trainable params: 0
_________________________________________________________________
Epoch 1/150
480/480 - 44s - loss: 1.6958 - accuracy: 0.3418 - val_loss: 2.1003 - val_accuracy: 0.1943
Epoch 2/150
480/480 - 41s - loss: 1.3527 - accuracy: 0.4923 - val_loss: 2.2971 - val_accuracy: 0.1826
Epoch 3/150
480/480 - 39s - loss: 1.2290 - accuracy: 0.5603 - val_loss: 1.9510 - val_accuracy: 0.3008
Epoch 4/150
480/480 - 38s - loss: 1.1569 - accuracy: 0.5819 - val_loss: 1.8383 - val_accuracy: 0.3838
Epoch 5/150
480/480 - 39s - loss: 1.1131 - accuracy: 0.5990 - val_loss: 1.8654 - val_accuracy: 0.3